In [1]:
# import data science libraries
import numpy as np
import pandas as pd

import re

import os.path
from os import path

from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from sklearn.cluster import KMeans

import wrangle as wr
import preprocessing_permits as pr
import explore as ex
import model as mo

import warnings
warnings.filterwarnings("ignore")

In [2]:
# global setting for DataFrames and visualizations
pd.set_option("display.max_columns", None)
plt.rc("figure", figsize=(16, 8))
sns.set_palette("colorblind")

In [3]:
df = wr.acquire_building_permits()
print(f"""Our DataFrame contains {df.shape[0]:,} observations and {df.shape[1]} features.""")
df

Our DataFrame contains 8,382 observations and 29 features.


,survey_date,csa_code,cbsa_code,moncov,cbsa_name,one_unit_bldgs_est,one_unit_units_est,one_unit_value_est,two_units_bldgs_est,two_units_units_est,two_units_value_est,three_to_four_units_bldgs_est,three_to_four_units_units_est,three_to_four_units_value_est,five_or_more_units_bldgs_est,five_or_more_units_units_est,five_or_more_units_value_est,one_unit_bldgs_rep,one_unit_units_rep,one_unit_value_rep,two_units_bldgs_rep,two_units_units_rep,two_units_value_rep,three_to_four_units_bldgs_rep,three_to_four_units_units_rep,three_to_four_units_value_rep,five_or_more_units_bldgs_rep,five_or_more_units_units_rep,five_or_more_units_value_rep
0,2019,104.0,10580.0,True,Albany-Schenectady-Troy NY,1120.0,1120.0,309397.0,20.0,40.0,7644.0,12.0,45.0,6074.0,48.0,665.0,60456.0,984.0,984.0,268946.0,18.0,36.0,6544.0,12.0,45.0,6074.0,34.0,580.0,56469.0
1,2019,430.0,48260.0,False,Weirton-Steubenville WV-OH,25.0,25.0,5782.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,5782.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019,999.0,10180.0,False,Abilene TX,354.0,354.0,72824.0,8.0,16.0,2093.0,0.0,0.0,0.0,0.0,0.0,0.0,353.0,353.0,72596.0,8.0,16.0,2093.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019,566.0,49660.0,False,Youngstown-Warren-Boardman OH-PA,323.0,323.0,73182.0,2.0,4.0,407.0,1.0,3.0,467.0,0.0,0.0,0.0,234.0,234.0,50054.0,2.0,4.0,407.0,1.0,3.0,467.0,0.0,0.0,0.0
4,2019,558.0,48700.0,False,Williamsport PA,66.0,66.0,16215.0,6.0,12.0,1610.0,0.0,0.0,0.0,0.0,0.0,0.0,49.0,49.0,12095.0,6.0,12.0,1610.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8377,1997,6240.0,9999.0,False,Pine Bluff AR MSA,100.0,100.0,4638.0,10.0,20.0,405.0,0.0,0.0,0.0,1.0,96.0,1826.0,100.0,100.0,4638.0,10.0,20.0,405.0,0.0,0.0,0.0,1.0,96.0,1826.0
8378,1997,6280.0,9999.0,False,Pittsburgh PA MSA,4432.0,4432.0,551173.0,66.0,132.0,9580.0,52.0,194.0,12291.0,102.0,1306.0,65021.0,3705.0,3705.0,481092.0,60.0,120.0,9251.0,45.0,173.0,12002.0,81.0,1111.0,56757.0
8379,1997,6320.0,9999.0,False,Pittsfield MA MSA,97.0,97.0,11578.0,3.0,6.0,342.0,0.0,0.0,0.0,1.0,120.0,5750.0,80.0,80.0,9447.0,3.0,6.0,342.0,0.0,0.0,0.0,1.0,120.0,5750.0
8380,1997,6340.0,9999.0,False,Pocatello ID MSA,263.0,263.0,23778.0,8.0,16.0,870.0,20.0,80.0,3630.0,2.0,14.0,543.0,263.0,263.0,23778.0,8.0,16.0,870.0,20.0,80.0,3630.0,2.0,14.0,543.0


In [4]:
df = wr.prep_building_permits(df)
print(f"""Our DataFrame contains {df.shape[0]:,} observations and {df.shape[1]} features.""")
df

Our DataFrame contains 8,269 observations and 30 features.


,major_city,major_state,survey_date,csa_code,cbsa_code,moncov,one_unit_bldgs_est,one_unit_units_est,one_unit_value_est,two_units_bldgs_est,two_units_units_est,two_units_value_est,three_to_four_units_bldgs_est,three_to_four_units_units_est,three_to_four_units_value_est,five_or_more_units_bldgs_est,five_or_more_units_units_est,five_or_more_units_value_est,one_unit_bldgs_rep,one_unit_units_rep,one_unit_value_rep,two_units_bldgs_rep,two_units_units_rep,two_units_value_rep,three_to_four_units_bldgs_rep,three_to_four_units_units_rep,three_to_four_units_value_rep,five_or_more_units_bldgs_rep,five_or_more_units_units_rep,five_or_more_units_value_rep
0,Abilene,TX,1997,40.0,9999.0,False,215.0,215.0,25825.0,0.0,0.0,0.0,1.0,3.0,125.0,0.0,0.0,0.0,215.0,215.0,25825.0,0.0,0.0,0.0,1.0,3.0,125.0,0.0,0.0,0.0
1,Abilene,TX,1998,40.0,9999.0,False,250.0,250.0,32745.0,2.0,4.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,32745.0,2.0,4.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abilene,TX,1999,40.0,9999.0,False,179.0,179.0,25372.0,3.0,6.0,382.0,0.0,0.0,0.0,0.0,0.0,0.0,179.0,179.0,25372.0,3.0,6.0,382.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abilene,TX,2000,40.0,9999.0,False,164.0,164.0,22306.0,9.0,18.0,1140.0,0.0,0.0,0.0,15.0,192.0,10200.0,164.0,164.0,22306.0,9.0,18.0,1140.0,0.0,0.0,0.0,15.0,192.0,10200.0
4,Abilene,TX,2001,40.0,9999.0,False,172.0,172.0,25539.0,28.0,56.0,3286.0,41.0,164.0,7631.0,13.0,192.0,6333.0,172.0,172.0,25539.0,28.0,56.0,3286.0,41.0,164.0,7631.0,13.0,192.0,6333.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8264,Yuma,AZ,2015,999.0,49740.0,False,765.0,765.0,117198.0,0.0,0.0,0.0,1.0,3.0,106.0,0.0,0.0,0.0,765.0,765.0,117198.0,0.0,0.0,0.0,1.0,3.0,106.0,0.0,0.0,0.0
8265,Yuma,AZ,2016,999.0,49740.0,False,890.0,890.0,137493.0,0.0,0.0,0.0,2.0,7.0,712.0,0.0,0.0,0.0,824.0,824.0,131881.0,0.0,0.0,0.0,2.0,7.0,712.0,0.0,0.0,0.0
8266,Yuma,AZ,2017,999.0,49740.0,False,1005.0,1005.0,163723.0,2.0,4.0,245.0,6.0,24.0,3118.0,8.0,68.0,8986.0,993.0,993.0,161953.0,1.0,2.0,245.0,6.0,24.0,3118.0,8.0,68.0,8986.0
8267,Yuma,AZ,2018,999.0,49740.0,False,1011.0,1011.0,160289.0,2.0,4.0,262.0,0.0,0.0,0.0,0.0,0.0,0.0,1011.0,1011.0,160289.0,2.0,4.0,262.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.head(46)

,major_city,major_state,survey_date,csa_code,cbsa_code,moncov,one_unit_bldgs_est,one_unit_units_est,one_unit_value_est,two_units_bldgs_est,two_units_units_est,two_units_value_est,three_to_four_units_bldgs_est,three_to_four_units_units_est,three_to_four_units_value_est,five_or_more_units_bldgs_est,five_or_more_units_units_est,five_or_more_units_value_est,one_unit_bldgs_rep,one_unit_units_rep,one_unit_value_rep,two_units_bldgs_rep,two_units_units_rep,two_units_value_rep,three_to_four_units_bldgs_rep,three_to_four_units_units_rep,three_to_four_units_value_rep,five_or_more_units_bldgs_rep,five_or_more_units_units_rep,five_or_more_units_value_rep
0,Abilene,TX,1997,40.0,9999.0,False,215.0,215.0,25825.0,0.0,0.0,0.0,1.0,3.0,125.0,0.0,0.0,0.0,215.0,215.0,25825.0,0.0,0.0,0.0,1.0,3.0,125.0,0.0,0.0,0.0
1,Abilene,TX,1998,40.0,9999.0,False,250.0,250.0,32745.0,2.0,4.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,250.0,32745.0,2.0,4.0,180.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Abilene,TX,1999,40.0,9999.0,False,179.0,179.0,25372.0,3.0,6.0,382.0,0.0,0.0,0.0,0.0,0.0,0.0,179.0,179.0,25372.0,3.0,6.0,382.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Abilene,TX,2000,40.0,9999.0,False,164.0,164.0,22306.0,9.0,18.0,1140.0,0.0,0.0,0.0,15.0,192.0,10200.0,164.0,164.0,22306.0,9.0,18.0,1140.0,0.0,0.0,0.0,15.0,192.0,10200.0
4,Abilene,TX,2001,40.0,9999.0,False,172.0,172.0,25539.0,28.0,56.0,3286.0,41.0,164.0,7631.0,13.0,192.0,6333.0,172.0,172.0,25539.0,28.0,56.0,3286.0,41.0,164.0,7631.0,13.0,192.0,6333.0
5,Abilene,TX,2002,40.0,9999.0,False,153.0,153.0,23080.0,5.0,10.0,525.0,6.0,18.0,1000.0,0.0,0.0,0.0,153.0,153.0,23080.0,5.0,10.0,525.0,6.0,18.0,1000.0,0.0,0.0,0.0
6,Abilene,TX,2003,999.0,10180.0,False,199.0,199.0,29826.0,11.0,22.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0,191.0,191.0,29135.0,11.0,22.0,1439.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Abilene,TX,2004,999.0,10180.0,False,283.0,283.0,41533.0,4.0,8.0,611.0,0.0,0.0,0.0,0.0,0.0,0.0,280.0,280.0,41262.0,4.0,8.0,611.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Abilene,TX,2005,999.0,10180.0,False,310.0,310.0,48199.0,11.0,22.0,1639.0,0.0,0.0,0.0,1.0,14.0,1200.0,305.0,305.0,47711.0,10.0,20.0,1445.0,0.0,0.0,0.0,1.0,14.0,1200.0
9,Abilene,TX,2006,999.0,10180.0,False,333.0,333.0,58363.0,8.0,16.0,1394.0,0.0,0.0,0.0,1.0,16.0,1500.0,331.0,331.0,58147.0,7.0,14.0,1200.0,0.0,0.0,0.0,1.0,16.0,1500.0


In [6]:
df = pr.get_permits_model_df()
print(f"""Our modeling DataFrame contains {df.shape[0]:,} observations & {df.shape[1]} features""")
df.head(46)

Our modeling DataFrame contains 8,269 observations & 6 features


,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value
0,Abilene,TX,1997,0.0,0.0,0.0
1,Abilene,TX,1998,0.0,0.0,0.0
2,Abilene,TX,1999,0.0,0.0,0.0
3,Abilene,TX,2000,15.0,192.0,10200000.0
4,Abilene,TX,2001,13.0,192.0,6333000.0
5,Abilene,TX,2002,0.0,0.0,0.0
6,Abilene,TX,2003,0.0,0.0,0.0
7,Abilene,TX,2004,0.0,0.0,0.0
8,Abilene,TX,2005,1.0,14.0,1200000.0
9,Abilene,TX,2006,1.0,16.0,1500000.0


In [7]:
# df["city_state_high_density_value_delta_pct"] = (
#     df.sort_values(["year"])
#     .groupby(["city", "state"])[["total_high_density_value"]]
#     .pct_change()
# )

In [8]:
df.tail(46)

,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value
8223,Yuba City,CA,1997,3.0,28.0,1111000.0
8224,Yuba City,CA,1998,0.0,0.0,0.0
8225,Yuba City,CA,1999,15.0,126.0,6752000.0
8226,Yuba City,CA,2000,0.0,0.0,0.0
8227,Yuba City,CA,2001,0.0,0.0,0.0
8228,Yuba City,CA,2002,0.0,0.0,0.0
8229,Yuba City,CA,2003,12.0,148.0,10281000.0
8230,Yuba City,CA,2004,3.0,17.0,747000.0
8231,Yuba City,CA,2005,4.0,31.0,1602000.0
8232,Yuba City,CA,2006,6.0,36.0,2019000.0


In [9]:
(7485000.0 - 4566000.0) / 4566000.0

0.6392904073587385

In [10]:
(12492000.0 - 30583000.0) / 30583000.0

-0.5915377824281464

In [11]:
(1 + 2.034637) / (1 + 0.231085)

2.4650101333376657

In [12]:
df = pr.add_new_features(df)
print(f"""Our modeling DataFrame contains {df.shape[0]:,} observations & {df.shape[1]} features""")
df.head()

Our modeling DataFrame contains 8,269 observations & 15 features


,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,avg_units_per_bldg,value_per_bldg,value_per_unit,city_state_high_density_bldgs_delta_pct,city_state_high_density_units_delta_pct,city_state_high_density_value_delta_pct,market_volume,market_volume_delta_pct,ei
0,Abilene,TX,1997,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.054924e+10,NaN,NaN
1,Abilene,TX,1998,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.529787e+10,0.231085,NaN
2,Abilene,TX,1999,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2.609590e+10,0.031545,NaN
3,Abilene,TX,2000,15.0,192.0,10200000.0,12.800000,680000.000000,53125.000,inf,inf,inf,2.742204e+10,0.050818,inf
4,Abilene,TX,2001,13.0,192.0,6333000.0,14.769231,487153.846154,32984.375,-0.133333,0.0,-0.379118,2.913103e+10,0.062322,0.584458


In [13]:
df.groupby("year").total_high_density_value.sum()

year
1997    2.054924e+10
1998    2.529787e+10
1999    2.609590e+10
2000    2.742204e+10
2001    2.913103e+10
2002    3.169504e+10
2003    2.305050e+10
2004    2.723856e+10
2005    3.435308e+10
2006    3.549408e+10
2007    3.375194e+10
2008    2.566581e+10
2009    1.005604e+10
2010    1.165572e+10
2011    1.574184e+10
2012    2.569934e+10
2013    3.321102e+10
2014    3.958652e+10
2015    5.200240e+10
2016    4.928300e+10
2017    5.158824e+10
2018    5.336251e+10
2019    5.956964e+10
Name: total_high_density_value, dtype: float64

In [14]:
df = pr.filter_top_cities_building_permits(df)
print(f"""Our modeling DataFrame contains {df.shape[0]:,} observations & {df.shape[1]} features""")
df.tail()

Our modeling DataFrame contains 2,990 observations & 16 features


,city,state,year,total_high_density_bldgs,total_high_density_units,total_high_density_value,avg_units_per_bldg,value_per_bldg,value_per_unit,city_state_high_density_bldgs_delta_pct,city_state_high_density_units_delta_pct,city_state_high_density_value_delta_pct,market_volume,market_volume_delta_pct,ei,city_state
2985,York,PA,2015,3.0,53.0,3185000.0,17.666667,1.061667e+06,60094.339623,-0.666667,-0.320513,-0.449724,5.200240e+10,0.313639,0.418895,York_PA
2986,York,PA,2016,3.0,34.0,1566000.0,11.333333,5.220000e+05,46058.823529,0.000000,-0.358491,-0.508320,4.928300e+10,-0.052294,0.518810,York_PA
2987,York,PA,2017,15.0,83.0,10204000.0,5.533333,6.802667e+05,122939.759036,4.000000,1.441176,5.515964,5.158824e+10,0.046775,6.224796,York_PA
2988,York,PA,2018,2.0,26.0,2160000.0,13.000000,1.080000e+06,83076.923077,-0.866667,-0.686747,-0.788318,5.336251e+10,0.034393,0.204643,York_PA
2989,York,PA,2019,25.0,389.0,26196000.0,15.560000,1.047840e+06,67341.902314,11.500000,13.961538,11.127778,5.956964e+10,0.116320,10.864068,York_PA


In [15]:
(4.928300e+10 - 5.200240e+10) / 5.200240e+10

-0.05229374028890974

In [16]:
(1 + -0.508320) / (1 + -0.052294)

0.518810686014439

In [17]:
df.groupby("year").total_high_density_value.sum()

year
1997    1.750160e+10
1998    2.181240e+10
1999    2.247141e+10
2000    2.364117e+10
2001    2.528117e+10
2002    2.676417e+10
2003    1.932928e+10
2004    2.220160e+10
2005    2.798906e+10
2006    3.100532e+10
2007    2.960063e+10
2008    2.262328e+10
2009    8.549583e+09
2010    9.750298e+09
2011    1.377148e+10
2012    2.299483e+10
2013    2.987518e+10
2014    3.625187e+10
2015    4.709459e+10
2016    4.439847e+10
2017    4.659169e+10
2018    4.795771e+10
2019    5.239455e+10
Name: total_high_density_value, dtype: float64